# Run me

In [1]:
from trulens.core import TruSession
from trulens.apps.custom import instrument

In [2]:
from snowflake.snowpark.session import Session
from snowflake.core import Root

In [3]:
import pandas as pd
import json
import numpy as np

In [4]:
from trulens.apps.custom import TruCustomApp
from trulens.providers.cortex.provider import Cortex
from trulens.core import Feedback
from trulens.core import Select

In [5]:
from snowflake.cortex import Complete

In [6]:
# Configuration
NUM_CHUNKS = 3  # Number of chunks to retrieve
SLIDE_WINDOW = 7  # Number of last conversations to remember
CORTEX_SEARCH_DATABASE = "CC_QUICKSTART_CORTEX_SEARCH_DOCS_3"
CORTEX_SEARCH_SCHEMA = "DATA"
CORTEX_SEARCH_SERVICE = "CC_SEARCH_SERVICE_CS"
COLUMNS = [
    "chunk",
    "relative_path",
    "category"
]

In [7]:
# Snowflake Connection and Session
connection_params = {
    "account": "bqxccfa-rob58877",
    "user": "MOUSTAFAISMAIL",
    "password": "DLayihJ3:6JCz42",
    "warehouse": "COMPUTE_WH",
    "database": "CC_QUICKSTART_CORTEX_SEARCH_DOCS_3",
    "schema": "DATA",
    "role": "ACCOUNTADMIN",
}

In [8]:
# Get active Snowflake session
session = Session.builder.configs(connection_params).create()
root = Root(session)
svc = root.databases[CORTEX_SEARCH_DATABASE].schemas[CORTEX_SEARCH_SCHEMA].cortex_search_services[CORTEX_SEARCH_SERVICE]

In [9]:
# Start Trulens session
tru_session = TruSession()

🦑 Initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `TruSession` to prevent this.


# Josh

In [32]:
from typing import List

class CortexSearchRetriever:

    def __init__(self, snowpark_session: Session, limit_to_retrieve: int = 4):
        self._snowpark_session = snowpark_session
        self._limit_to_retrieve = NUM_CHUNKS

    @instrument
    def retrieve(self, query: str, category: str) -> List[str]:
        root = Root(self._snowpark_session)

        cortex_search_service = (
            root.databases[CORTEX_SEARCH_DATABASE]
            .schemas[CORTEX_SEARCH_SCHEMA]
            .cortex_search_services[CORTEX_SEARCH_SERVICE]
        )
        
        filter_obj = {"@eq": {"category": category}}
        response = cortex_search_service.search(
                query, 
                columns = COLUMNS,
                filter=filter_obj,
                limit=self._limit_to_retrieve,
        )

        # return response
        
        if response.results:
            return [curr["chunk"] for curr in response.results]
        else:
            return []

decorating <function CortexSearchRetriever.retrieve at 0x72e474874c20>
adding method <class '__main__.CortexSearchRetriever'> retrieve __main__


In [33]:
retriever = CortexSearchRetriever(snowpark_session=session, limit_to_retrieve=4)
retrieved_context = retriever.retrieve(query="How to make chicken peas?", category="Juices")
retrieved_context

['1 tsp Cooking Time: 5 minutes Steps: Blend kiwi and spinach with honey, strain, and serve chilled. Cuisine: General Diet Type: Vegan  \\nRecipe ID: 922  \\nRecipe Name: Pineapple Mint Splash Ingredients: Pineapple, Mint, Lemon Quantities (for 1 person): Pineapple: 1/2 cup (cubed), Mint: 5 leaves, Lemon: 1/2 (juiced) Cooking Time: 5 minutes Steps: Blend pineapple and mint, strain, mix with lemon juice, and serve chilled. Cuisine: Tropical Diet Type: Vegan  \\nRecipe ID: 923  \\nRecipe Name: Apple Cucumber Cooler Ingredients: Apple, Cucumber, Lemon Quantities (for 1 person): Apple: 1 (juiced), Cucumber: 1/2 (juiced), Lemon: 1/2 (juiced) Cooking Time: 5 minutes Steps: Mix apple and cucumber juice, add lemon juice, and serve chilled. Cuisine: General Diet Type: Vegan  \\nRecipe ID: 924  \\nRecipe Name: Strawberry Basil Refresher Ingredients: Strawberry, Basil, Honey Quantities (for 1 person): Strawberry: 1/2 cup (chopped), Basil: 5 leaves, Honey: 1 tsp Cooking Time: 5 minutes Steps: Blen

In [36]:
from snowflake.snowpark.context import get_active_session

snow_session = get_active_session()

In [39]:
from trulens.providers.cortex.provider import Cortex
from trulens.core import Feedback
from trulens.core import Select
import numpy as np

provider = Cortex(session, "mistral-large2")

f_context_relevance = (
    Feedback(provider.context_relevance, name="Context Relevance")
    .on_input_output()
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.main_output or `Select.RecordOutput` .


In [40]:
f_context_relevance("How to make chicken peas?", retrieved_context)

0.0

In [41]:
from trulens.apps.custom import TruCustomApp

tru_retriever = TruCustomApp(
    retriever,
    app_name="FOMC Retriever",
    app_version="base",
    feedbacks=[f_context_relevance],
)

Function <function CortexSearchRetriever.retrieve at 0x72e4749cb240> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.CortexSearchRetriever object at 0x72e474ea7450> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function CortexSearchRetriever.retrieve at 0x72e474875a80> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.CortexSearchRetriever object at 0x72e474ea7450> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function CortexSearchRetriever.retrieve at 0x72e4749cb4c0> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.CortexSearchRetriever object at 0x72e474ea7450> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function CortexSearchRetriever.retrieve at 0x72e47497c900> w

instrumenting <class '__main__.CortexSearchRetriever'> for base <class '__main__.CortexSearchRetriever'>
	instrumenting retrieve
skipping base <class 'object'> because of class


In [43]:
with tru_retriever as recording:
    retrieved_documents = retriever.retrieve("How to make vanilla milke shake?", category="Juices")

retrieved_documents

Could not find an instance of DummyEndpoint. trulens will create an endpoint for cost tracking.


calling <function CortexSearchRetriever.retrieve at 0x72e474874c20> with (<__main__.CortexSearchRetriever object at 0x72e474ea7450>, 'How to make vanilla milke shake?')


Unsure what the main input string is for the call to retrieve with args <BoundArguments (self=<__main__.CortexSearchRetriever object at 0x72e474ea7450>, query='How to make vanilla milke shake?', category='Juices')>.


['and serve chilled. Cuisine: General Diet Type: Vegan  \\nRecipe ID: 894  \\nRecipe Name: Pear Vanilla Delight Ingredients: Pear, Vanilla Extract, Honey Quantities (for 1 person): Pear: 1 (peeled and chopped), Vanilla Extract: 1/4 tsp, Honey: 1 tsp Cooking Time: 5 minutes Steps: Blend pear with vanilla extract and honey, strain, and serve chilled. Cuisine: General Diet Type: Vegan  \\nRecipe ID: 895  \\nRecipe Name: Cherry Lime Fizz Ingredients: Cherry, Lime, Sparkling Water Quantities (for 1 person): Cherry: 1/2 cup (pitted), Lime: 1/2 (juiced), Sparkling Water: 1/2 cup Cooking Time: 5 minutes Steps: Blend cherries, strain, mix with lime juice and sparkling water, and serve chilled. Cuisine: General Diet Type: Vegan  \\nRecipe ID: 896  \\nRecipe Name: Pineapple Ginger Cooler Ingredients: Pineapple, Ginger, Coconut Water Quantities (for 1 person): Pineapple: 1/2 cup (cubed), Ginger: 1/4 tsp (grated), Coconut Water: 1/2 cup Cooking Time: 5 minutes Steps: Blend pineapple and ginger, str

In [45]:
tru_session.get_leaderboard()

,,Context Relevance,latency,total_cost
app_name,app_version,,,
FOMC Retriever,base,0.0,1.133732,0.0


---------------

# My Trulens

#### Create the class 

In [8]:
from typing import List

class MyCortexSearchRetriever:

    @instrument
    def get_similar_chunks_search_service(self, query, category):
        """Search for similar chunks based on query and category."""
        if category == "ALL":
            response = svc.search(query, COLUMNS, limit=NUM_CHUNKS)
        else:
            filter_obj = {"@eq": {"category": category}}
            response = svc.search(
                query,
                COLUMNS,
                filter=filter_obj,
                limit=NUM_CHUNKS,
            )
        # st.sidebar.json(response.json())
        if response.results:
            output = [curr["chunk"] for curr in response.results]
            return response.json(), output

        else:
            output = []
            return response.json(), output

decorating <function MyCortexSearchRetriever.get_similar_chunks_search_service at 0x77c042a02c00>
adding method <class '__main__.MyCortexSearchRetriever'> get_similar_chunks_search_service __main__


#### Instantiate the class or app

In [9]:
myretriever = MyCortexSearchRetriever()

In [10]:
myretriever = MyCortexSearchRetriever()
my_retrieved_context = myretriever.get_similar_chunks_search_service(query="How to make vanilla milk shake?", category="Juices")

/tmp/ipykernel_894/3009735916.py:21: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  return response.json(), output


In [11]:
my_retrieved_context

('{"results":[{"chunk":"and serve chilled. Cuisine: General Diet Type: Vegan  \\\\nRecipe ID: 894  \\\\nRecipe Name: Pear Vanilla Delight Ingredients: Pear, Vanilla Extract, Honey Quantities (for 1 person): Pear: 1 (peeled and chopped), Vanilla Extract: 1/4 tsp, Honey: 1 tsp Cooking Time: 5 minutes Steps: Blend pear with vanilla extract and honey, strain, and serve chilled. Cuisine: General Diet Type: Vegan  \\\\nRecipe ID: 895  \\\\nRecipe Name: Cherry Lime Fizz Ingredients: Cherry, Lime, Sparkling Water Quantities (for 1 person): Cherry: 1/2 cup (pitted), Lime: 1/2 (juiced), Sparkling Water: 1/2 cup Cooking Time: 5 minutes Steps: Blend cherries, strain, mix with lime juice and sparkling water, and serve chilled. Cuisine: General Diet Type: Vegan  \\\\nRecipe ID: 896  \\\\nRecipe Name: Pineapple Ginger Cooler Ingredients: Pineapple, Ginger, Coconut Water Quantities (for 1 person): Pineapple: 1/2 cup (cubed), Ginger: 1/4 tsp (grated), Coconut Water: 1/2 cup Cooking Time: 5 minutes Step

#### Feedback function

In [12]:
from trulens.providers.cortex.provider import Cortex
from trulens.core import Feedback
from trulens.core import Select
import numpy as np

provider = Cortex(session, "mistral-large2")

f_context_relevance = (
    Feedback(provider.context_relevance, name="Context Relevance")
    .on(Select.RecordCalls.get_similar_chunks_search_service.args['query'])
    .on(Select.RecordCalls.get_similar_chunks_search_service.rets.collect())
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.app.get_similar_chunks_search_service.args.query .
✅ In Context Relevance, input context will be set to __record__.app.get_similar_chunks_search_service.rets.collect() .


In [13]:
f_context_relevance("How to make vanilla milk shake?", my_retrieved_context)

0.3333333333333333

#### Register the app

In [14]:
from trulens.apps.custom import TruCustomApp

my_tru_retriever = TruCustomApp(
    myretriever,
    app_name="My Retriever",
    app_version="base",
    feedbacks=[f_context_relevance],
)

instrumenting <class '__main__.MyCortexSearchRetriever'> for base <class '__main__.MyCortexSearchRetriever'>
	instrumenting get_similar_chunks_search_service
skipping base <class 'object'> because of class


In [15]:
with my_tru_retriever as recording:
    retrieved_documents = myretriever.get_similar_chunks_search_service(query="How to make cocunut milk?", category="Juices")

Could not find an instance of DummyEndpoint. trulens will create an endpoint for cost tracking.


calling <function MyCortexSearchRetriever.get_similar_chunks_search_service at 0x77c042a02c00> with (<__main__.MyCortexSearchRetriever object at 0x77c043332b90>,)


/tmp/ipykernel_894/3009735916.py:21: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  return response.json(), output
Unsure what the main input string is for the call to get_similar_chunks_search_service with args <BoundArguments (self=<__main__.MyCortexSearchRetriever object at 0x77c043332b90>, query='How to make cocunut milk?', category='Juices')>.


In [16]:
retrieved_documents

('{"results":[{"chunk":"coconut milk until smooth. Serve immediately. Cuisine: General Diet Type: Vegan  \\\\nRecipe ID: 557  \\\\nRecipe Name: Pineapple Coconut Cooler Ingredients: Pineapple, Coconut water, Lime Quantities (for 1 person): Pineapple: 1/2 cup (cubed), Coconut water: 1/2 cup, Lime: 1/2 Cooking Time: 5 minutes Steps: Blend pineapple with coconut water, add lime juice, and serve chilled. Cuisine: General Diet Type: Vegan  \\\\nRecipe ID: 558  \\\\nRecipe Name: Mixed Berry Sparkler Ingredients: Mixed berries (strawberries, blueberries, raspberries), Lemon, Soda water Quantities (for 1 person): Mixed berries: 1/2 cup, Lemon: 1/2, Soda water: 1/2 cup Cooking Time: 5 minutes Steps: Blend mixed berries, strain, add lemon juice, and top with soda water. Serve cold. Cuisine: General Diet Type: Vegan  \\\\nRecipe ID: 559  \\\\nRecipe Name: Honeydew Mint Cooler Ingredients: Honeydew melon, Mint leaves, Lime Quantities (for 1 person): Honeydew melon: 1/2 cup (cubed), Mint leaves: 4-

In [17]:
tru_session.get_leaderboard()

,,Context Relevance,latency,total_cost
app_name,app_version,,,
RAG,simple,0.333333,0.609406,0.0
My Retriever,base,0.133333,0.564138,0.0
FOMC Retriever,base,0.000000,1.133732,0.0


In [18]:
my_tru_retriever.print_instrumented

<bound method App.print_instrumented of TruCustomApp(tru_class_info=trulens.apps.custom.TruCustomApp, app_id='app_hash_84a87cac3345064b3a989607f4da64e0', app_name='My Retriever', app_version='base', tags='-', metadata={}, feedback_definitions=['feedback_definition_hash_388f492b63577414ec7b816ff1cbe398'], feedback_mode=<FeedbackMode.WITH_APP_THREAD: 'with_app_thread'>, record_ingest_mode=<RecordIngestMode.IMMEDIATE: 'immediate'>, root_class=__main__.MyCortexSearchRetriever, app=<__main__.MyCortexSearchRetriever object at 0x77c043332b90>, initial_app_loader_dump=None, app_extra_json={}, feedbacks=[FeedbackDefinition(Context Relevance,
	selectors={'question': Lens().__record__.app.get_similar_chunks_search_service.args.query, 'context': Lens().__record__.app.get_similar_chunks_search_service.rets.collect()},
	if_exists=None
)], session=TruSession(RETRY_RUNNING_SECONDS=60.0, RETRY_FAILED_SECONDS=300.0, DEFERRED_NUM_RUNS=32, RECORDS_BATCH_TIMEOUT_IN_SEC=10, GROUND_TRUTHS_BATCH_SIZE=100, con

-----

# RAG app

In [1]:
from trulens.core import TruSession
from trulens.apps.custom import instrument

In [2]:
from snowflake.snowpark.session import Session
from snowflake.core import Root

In [3]:
import pandas as pd
import json
import numpy as np

In [5]:
# Configuration
NUM_CHUNKS = 3  # Number of chunks to retrieve
SLIDE_WINDOW = 7  # Number of last conversations to remember
CORTEX_SEARCH_DATABASE = "CC_QUICKSTART_CORTEX_SEARCH_DOCS_3"
CORTEX_SEARCH_SCHEMA = "DATA"
CORTEX_SEARCH_SERVICE = "CC_SEARCH_SERVICE_CS"
COLUMNS = [
    "chunk",
    "relative_path",
    "category"
]

In [6]:
# Snowflake Connection and Session
connection_params = {
    "account": "bqxccfa-rob58877",
    "user": "MOUSTAFAISMAIL",
    "password": "DLayihJ3:6JCz42",
    "warehouse": "COMPUTE_WH",
    "database": "CC_QUICKSTART_CORTEX_SEARCH_DOCS_3",
    "schema": "DATA",
    "role": "ACCOUNTADMIN",
}

In [7]:
# Get active Snowflake session
session = Session.builder.configs(connection_params).create()
root = Root(session)
svc = root.databases[CORTEX_SEARCH_DATABASE].schemas[CORTEX_SEARCH_SCHEMA].cortex_search_services[CORTEX_SEARCH_SERVICE]

In [8]:
# Start Trulens session
tru_session = TruSession()

🦑 Initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `TruSession` to prevent this.


In [48]:
class RAG:

    def __init__(self):
        self.myretriever = MyCortexSearchRetriever()
        
    @instrument
    def retrieve_context(self, query: str, category: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        return self.myretriever.get_similar_chunks_search_service(query, category)
    

    def create_prompt(self, query, category):
        """Create a prompt for the LLM with context from search results and chat history."""

        # if st.session_state.use_chat_history:
        #     chat_history = get_chat_history()
        #     if chat_history:
        #         question_summary = summarize_question_with_history(chat_history, query)
        #         prompt_context = rag.get_similar_chunks_search_service(question_summary, category)
        #     else:
        #         prompt_context = rag.get_similar_chunks_search_service(query, category)
        # else:
        #     prompt_context = rag.get_similar_chunks_search_service(query, category)
        #     chat_history = ""

        prompt_context, string_output = self.retrieve_context(query, category)

        prompt = f"""
        I am Ali, a friendly and witty chef who specializes in {category} recipes! I love helping people cook and finding the perfect recipes from our collection.

        Conversation Flow:
        1. When suggesting recipes:
            - Prioritize recipes that makes use of all ingredients
            - First list all matching recipes as numbered options
            - Ask which recipe they'd like to know more about
        2. When user selects a recipe, provide full details in this format:
            Recipe Name:
            Quantities (for 1 person):
            Cooking Time:
            Steps:
            Cuisine:
            General Diet Type:

        
        <context>
        {prompt_context}
        </context>

        User Query: {query}
        Current Category: {category}

        Response (as Ali, friendly and category-aware):
        """

        json_data = json.loads(prompt_context)
        relative_paths = set(item.get('relative_path', '') for item in json_data['results'])
        return prompt, relative_paths


    @instrument
    def complete_query(self, query, category):
        """Complete the query using Snowflake Cortex with Mistral model."""
        prompt, relative_paths = self.create_prompt(query, category)
        cmd = """
            select snowflake.cortex.complete(?, ?) as response
        """
        df_response = session.sql(cmd, params=['mistral-large', prompt]).collect()
        resp = df_response[0].as_dict()['RESPONSE']
        return df_response, relative_paths


    @instrument
    def query(self, query: str, category: str) -> str:
        context_str = self.retrieve_context(query, category)
        return self.complete_query(query, context_str)

decorating <function RAG.retrieve_context at 0x7700001bdc60>
decorating <function RAG.complete_query at 0x76fff046c4a0>
decorating <function RAG.query at 0x76fff046d8a0>
adding method <class '__main__.RAG'> retrieve_context __main__
adding method <class '__main__.RAG'> complete_query __main__
adding method <class '__main__.RAG'> query __main__


In [49]:
full_rag = RAG()

In [12]:
full_rag = RAG()
p = full_rag.complete_query(query="how to make chicken with peas?", category="MainCourse")
print(p)

/tmp/ipykernel_709/3009735916.py:21: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  return response.json(), output


[Row(RESPONSE=' I found a couple of recipes that match your query. Here are your options:\n        1. Chicken and Rice Skillet: This recipe includes chicken breast, rice, bell pepper, and chicken broth along with peas.\n        2. Egg Fried Rice: This is an Asian dish with rice, eggs, soy sauce, and vegetables like peas and carrot.\n\n        Which recipe would you like to know more about?')]


In [20]:
full_rag.create_prompt(query="how to make chicken with peas?", category="MainCourse")

/tmp/ipykernel_709/3009735916.py:21: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  return response.json(), output


('\n        I am Ali, a friendly and witty chef who specializes in MainCourse recipes! I love helping people cook and finding the perfect recipes from our collection.\n\n        Conversation Flow:\n        1. When suggesting recipes:\n            - Prioritize recipes that makes use of all ingredients\n            - First list all matching recipes as numbered options\n            - Ask which recipe they\'d like to know more about\n        2. When user selects a recipe, provide full details in this format:\n            Recipe Name:\n            Quantities (for 1 person):\n            Cooking Time:\n            Steps:\n            Cuisine:\n            General Diet Type:\n\n        \n        <context>\n        {"results":[{"chunk":"Fried Rice Ingredients: Rice, Carrot, Peas, Soy sauce, Olive oil, Garlic, Green onions Quantities (for 1 person): Rice: 1/2 cup (cooked), Carrot: 1/2 (diced), Peas: 1/4 cup, Soy sauce: 1 tbsp, Olive oil: 1 tsp, Garlic: 1 clove (minced), Green onions: 1 (chopped

In [ ]:
print(p[0][0].as_dict()['RESPONSE'])

In [ ]:
f_context_relevance = (
    Feedback(provider.context_relevance, name="Context Relevance")
    .on(Select.RecordCalls.get_similar_chunks_search_service.args['query'])
    .on(Select.RecordCalls.get_similar_chunks_search_service.rets.collect())
    .aggregate(np.mean)
)

In [51]:
from functools import partial
from trulens.providers.cortex.provider import Cortex
from trulens.core import Feedback
from trulens.core import Select
import numpy as np


provider = Cortex(session, "mistral-large2")

f_groundedness = (
    Feedback(partial(provider.groundedness_measure_with_cot_reasons,
            use_sent_tokenize=False), name="Groundedness")
    .on(Select.RecordCalls.retrieve_context.rets.collect())
    .on_output()
)

f_context_relevance = (
    Feedback(provider.context_relevance, name="Context Relevance")
    .on_input()
    .on(Select.RecordCalls.retrieve_context.rets.collect())
    .aggregate(np.mean)
)

f_answer_relevance = (
    Feedback(provider.relevance, name="Answer Relevance")
    .on_input()
    .on_output()
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.app.retrieve_context.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.retrieve_context.rets.collect() .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [52]:
from trulens.apps.custom import TruCustomApp

full_tru_rag = TruCustomApp(
    full_rag,
    app_name="RAG",
    app_version="simple",
    feedbacks=[
        f_groundedness,
        f_answer_relevance,
        f_context_relevance
        ],
    )

Function <function RAG.retrieve_context at 0x770010c8d800> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG object at 0x770000bc94d0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG.retrieve_context at 0x770000186e80> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG object at 0x770000bc94d0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG.complete_query at 0x76fff37f5a80> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG object at 0x770000bc94d0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG.retrieve_context at 0x76fff37f56c0> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__

instrumenting <class '__main__.MyCortexSearchRetriever'> for base <class '__main__.MyCortexSearchRetriever'>
	instrumenting get_similar_chunks_search_service
skipping base <class 'object'> because of class
instrumenting <class '__main__.RAG'> for base <class '__main__.RAG'>
	instrumenting retrieve_context
	instrumenting complete_query
	instrumenting query
skipping base <class 'object'> because of class


In [53]:
with full_tru_rag as recording:
    response = full_rag.query(query="how to make chicken with peas?", category="MainCourse")

response

calling <function RAG.query at 0x76fff046d8a0> with (<__main__.RAG object at 0x770000bc94d0>,)
calling <function RAG.retrieve_context at 0x7700001bdc60> with (<__main__.RAG object at 0x770000bc94d0>, 'how to make chicken with peas?', 'MainCourse')
calling <function MyCortexSearchRetriever.get_similar_chunks_search_service at 0x770010c8de40> with (<__main__.MyCortexSearchRetriever object at 0x770001455590>, 'how to make chicken with peas?', 'MainCourse')


/tmp/ipykernel_709/3009735916.py:21: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  return response.json(), output
Error calling wrapped function get_similar_chunks_search_service.
snowflake.core.exceptions.APIError: (400)
Reason: Bad Request
Error Message: Invalid filter query: Invalid argument: Expected a string but got ["{\"results\":[{\"chunk\":\"Fried Rice Ingredients: Rice, Carrot, Peas, Soy sauce, Olive oil, Garlic, Green onions Quantities (for 1 person): Rice: 1/2 cup (cooked), Carrot: 1/2 (diced), Peas: 1/4 cup, Soy sauce: 1 tbsp, Olive oil: 1 tsp, Garlic: 1 clove (minced), Green onions: 1 (chopped Cooking Time: 15 minutes Steps: Heat olive oil in a pan, add garlic and cook for 1 minute. Add carrot and peas, cook for 3-4 minutes. Add cooked rice and soy sauce, stir-fry for another 5 minutes. Garnish with

calling <function RAG.complete_query at 0x76fff046c4a0> with (<__main__.RAG object at 0x770000bc94d0>, 'how to make chicken with peas?', ('{"results":[{"chunk":"Fried Rice Ingredients: Rice, Carrot, Peas, Soy sauce, Olive oil, Garlic, Green onions Quantities (for 1 person): Rice: 1/2 cup (cooked), Carrot: 1/2 (diced), Peas: 1/4 cup, Soy sauce: 1 tbsp, Olive oil: 1 tsp, Garlic: 1 clove (minced), Green onions: 1 (chopped Cooking Time: 15 minutes Steps: Heat olive oil in a pan, add garlic and cook for 1 minute. Add carrot and peas, cook for 3-4 minutes. Add cooked rice and soy sauce, stir-fry for another 5 minutes. Garnish with green onions. Serve hot. Cuisine: Asian Diet Type: Vegan  \\\\nRecipe ID: 332  \\\\nRecipe Name: Chicken and Rice Skillet Ingredients: Chicken breast, Rice, Bell pepper, Olive oil, Chicken broth, Salt, Pepper Quantities (for 1 person): Chicken breast: 1 (150 grams), Rice: 1/2 cup (uncooked), Bell pepper: 1/2 (sliced), Olive oil: 1 tsp, Chicken broth: 1/2 cup, Salt:

APIError: (400)
Reason: Bad Request
Error Message: Invalid filter query: Invalid argument: Expected a string but got ["{\"results\":[{\"chunk\":\"Fried Rice Ingredients: Rice, Carrot, Peas, Soy sauce, Olive oil, Garlic, Green onions Quantities (for 1 person): Rice: 1/2 cup (cooked), Carrot: 1/2 (diced), Peas: 1/4 cup, Soy sauce: 1 tbsp, Olive oil: 1 tsp, Garlic: 1 clove (minced), Green onions: 1 (chopped Cooking Time: 15 minutes Steps: Heat olive oil in a pan, add garlic and cook for 1 minute. Add carrot and peas, cook for 3-4 minutes. Add cooked rice and soy sauce, stir-fry for another 5 minutes. Garnish with green onions. Serve hot. Cuisine: Asian Diet Type: Vegan  \\\\nRecipe ID: 332  \\\\nRecipe Name: Chicken and Rice Skillet Ingredients: Chicken breast, Rice, Bell pepper, Olive oil, Chicken broth, Salt, Pepper Quantities (for 1 person): Chicken breast: 1 (150 grams), Rice: 1/2 cup (uncooked), Bell pepper: 1/2 (sliced), Olive oil: 1 tsp, Chicken broth: 1/2 cup, Salt: pinch, Pepper:

In [46]:
tru_session.get_leaderboard()

,,Context Relevance,latency,total_cost
app_name,app_version,,,
RAG,simple,0.333333,0.609406,0.0
My Retriever,base,0.133333,0.594773,0.0
FOMC Retriever,base,0.000000,1.133732,0.0


---

# Replicating Documentation

In [11]:
from snowflake.cortex import Complete

class ragclass:

    @instrument
    def retrieve(self, query: str) -> list:
        returned_cortex = svc.search(query, columns=COLUMNS, limit=NUM_CHUNKS)

        json_data = json.loads(returned_cortex.model_dump_json())
        relative_paths = set(item.get('relative_path', '') for item in json_data['results'])

        # print(json_data, "\n--------------------")
        # print(relative_paths)

        if returned_cortex.results:
            return [curr["chunk"] for curr in returned_cortex.results]
    
    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:

        if len(context_str) == 0:
            return "Sorry, I couldn't find an answer to your question."

        category = "Juices"
        
        prompt = f"""
        I am Ali, a friendly and witty chef who specializes in {category} recipes! I love helping people cook and finding the perfect recipes from our collection.

        Conversation Flow:
        1. When suggesting recipes:
            - Prioritize recipes that makes use of all ingredients
            - First list all matching recipes as numbered options
            - Ask which recipe they'd like to know more about
        2. When user selects a recipe, provide full details in this format:
            Recipe Name:
            Quantities (for 1 person):
            Cooking Time:
            Steps:
            Cuisine:
            General Diet Type:

        <context>
        {context_str}
        </context>

        User Query: {query}
        Current Category: {category}

        Response (as Ali, friendly and category-aware):
        """

        cmd = """
            select snowflake.cortex.complete(?, ?) as response
        """

        return Complete("mistral-large2", prompt)
        # df_response = session.sql(cmd, params=['mistral-large', prompt]).collect()

        # df_response_string = df_response[0].as_dict()['RESPONSE']
        # return {'response': df_response_string}
    
    @instrument
    def query(self, query: str):
        context_str = self.retrieve(query=query)
        completion = self.generate_completion(
            query=query, context_str=context_str
        )

        return completion

decorating <function ragclass.retrieve at 0x78c261d763e0>
decorating <function ragclass.generate_completion at 0x78c261d77b00>
decorating <function ragclass.query at 0x78c261d77a60>
adding method <class '__main__.ragclass'> retrieve __main__
adding method <class '__main__.ragclass'> generate_completion __main__
adding method <class '__main__.ragclass'> query __main__


In [12]:
myrag = ragclass()

In [11]:
m = myrag.retrieve("vanilla milk")
print(m)

['Cream Ingredients: Banana, almond milk, vanilla extract Quantities (for 1 person): Banana: 1, almond milk: 1/4 cup, vanilla extract: 1/2 tsp Cooking Time: 5 minutes (plus freezing time) Steps: Blend frozen banana with almond milk and vanilla until smooth. Freeze for 1 hour before serving. Cuisine: General Diet Type: Vegan  \\nRecipe ID: 1803 Recipe Name: Chocolate Avocado Mousse Ingredients: Avocado, cocoa powder, maple syrup, vanilla extract Quantities (for 1 person): Avocado: 1/2, cocoa powder: 2 tbsp, maple syrup: 2 tbsp, vanilla extract: 1/2 tsp Cooking Time: 5 minutes Steps: Blend avocado, cocoa powder, maple syrup, and vanilla extract until smooth. Refrigerate for 1 hour. Cuisine: General Diet Type: Vegan  \\nRecipe ID: 1804 Recipe Name: Mango Coconut Sorbet Ingredients: Mango, coconut milk, honey  \\nQuantities (for 1 person): Mango: 1/2, coconut milk: 1/2 cup, honey: 1 tbsp Cooking Time: 5 minutes (plus freezing time) Steps: Blend mango, coconut milk, and honey until smooth.'

In [12]:
# from functools import partial
from trulens.providers.cortex.provider import Cortex
from trulens.core import Feedback
from trulens.core import Select
# import numpy as np

provider = Cortex(session, "mistral-large2")

# Define a groundedness feedback function
f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons, name="Groundedness"
    )
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
)
# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name="Answer Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)

# Context relevance between question and each context chunk.
f_context_relevance = (
    Feedback(
        provider.context_relevance_with_cot_reasons, name="Context Relevance"
    )
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)  # choose a different aggregation method if you wish
)

✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.app.retrieve.args.query .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.app.retrieve.args.query .
✅ In Context Relevance, input context will be set to __record__.app.retrieve.rets.collect() .


In [ ]:
tru_rag = TruCustomApp(
    myrag,
    app_name="rag-documentation",
    app_version="base",
    feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance],
)

instrumenting <class '__main__.ragclass'> for base <class '__main__.ragclass'>
	instrumenting retrieve
	instrumenting generate_completion
	instrumenting query
skipping base <class 'object'> because of class


In [74]:
tru_rag.main_input()

TypeError: App.main_input() missing 3 required positional arguments: 'func', 'sig', and 'bindings'

In [19]:
p = myrag.query("How to make chocolate milk?")

{'results': [{'chunk': 'milk, and melted butter in a microwave-safe mug. Sprinkle chocolate chips on top. Microwave for 1-2 minutes until cooked. Serve warm. Cuisine: General Diet Type: Vegetarian  \\nRecipe ID: 1539 Recipe Name: Caramelized Banana Ingredients: Banana, brown sugar, butter Quantities (for 1 person): Banana: 1 (sliced lengthwise), brown sugar: 1 tsp, butter: 1/2 tsp Cooking Time: 10 minutes Steps: Heat butter in a pan, sprinkle brown sugar, and place banana slices. Cook until caramelized. Serve warm. Cuisine: General Diet Type: Vegetarian  \\nRecipe ID: 1540 Recipe Name: Strawberry Cream Cup Ingredients: Fresh strawberries, whipped cream, sugar Quantities (for 1 person): Strawberries: 1/2 cup (sliced), whipped cream: 1/4 cup, sugar: 1 tsp Cooking Time: 5 minutes Steps: Layer sliced strawberries and whipped cream in a cup. Sprinkle sugar on top. Serve chilled. Cuisine: General Diet Type: Vegetarian  \\nRecipe ID: 1541 Recipe Name: Mango Coconut Bowl Ingredients: Fresh man

In [26]:
p[0].as_dict()['RESPONSE']

" I'm sorry, but I don't have a recipe for chocolate milk in my collection. However, I can suggest some delicious juice recipes if you'd like! How about a refreshing strawberry cream cup or a tropical mango coconut bowl?\n\n        For the Strawberry Cream Cup, you'll need:\n        - 1/2 cup sliced fresh strawberries\n        - 1/4 cup whipped cream\n        - 1 tsp sugar\n\n        Just layer the sliced strawberries and whipped cream in a cup, sprinkle sugar on top, and serve chilled. It's a quick and easy treat that's perfect for any time of day!\n\n        Or, if you're in the mood for something a little more exotic, the Mango Coconut Bowl might be just what you're looking for. Here's what you'll need:\n        - Fresh mango\n        - Coconut milk\n        - Shredded coconut\n\n        Simply blend the mango and coconut milk together, pour it into a bowl, and top with shredded coconut. It's a delicious and healthy snack that's sure to satisfy your sweet tooth!\n\n        Which rec

In [19]:
t = myrag.query("How to make pina colada")
print(t)

KeyboardInterrupt: 

In [67]:
x = myrag.generate_completion(query="How to make pina colada", context_str="He wants to know a recipe")

In [68]:
x

{'response': " Sure, I have a couple of Pina Colada recipes for you!\n\n        1. Classic Pina Colada\n        2. Virgin Pina Colada\n        3. Pina Colada Smoothie\n\n        Could you please tell me which one you'd like to know more about?"}

In [45]:
print(myrag.query("How to make pina colada"))

 Sure, I have a great recipe for Pina Colada Smoothie! Here are the details:

        Recipe Name: Pina Colada Smoothie
        Quantities (for 1 person):
        - Pineapple: 1/2 cup
        - Coconut milk: 1/2 cup
        - Banana: 1/2
        Cooking Time: 2 minutes
        Steps: Blend pineapple, coconut milk, and banana until smooth. Serve chilled.
        Cuisine: General
        General Diet Type: Vegan

        Would you like to know about any other Juices recipes?


In [18]:
context = Select.RecordCalls.retrieve.rets.rets[:]
context.get()

TypeError: Lens.get() missing 1 required positional argument: 'obj'

In [ ]:
with tru_rag as recording:
    myrag.query("How to make pina colada")

Could not find an instance of DummyEndpoint. trulens will create an endpoint for cost tracking.


calling <function ragclass.query at 0x7a2742edc040> with (<__main__.ragclass object at 0x7a2742ee02d0>, 'How to make pina colada')
calling <function ragclass.retrieve at 0x7a2742ec3d80> with (<__main__.ragclass object at 0x7a2742ee02d0>,)


calling <function ragclass.generate_completion at 0x7a2742ec3f60> with (<__main__.ragclass object at 0x7a2742ee02d0>,)


/tmp/ipykernel_840/3177399192.py:56: DeprecationWarning: Complete() is deprecated and will be removed in a future release. Use complete() instead
  return Complete("mistral-large2", prompt)


/home/vscode/.local/lib/python3.11/site-packages/trulens/feedback/llm_provider.py:1521: UserWarning: Failed to process and remove trivial statements. Proceeding with all statements.
  warnings.warn(


In [16]:
tru_session.get_leaderboard(app_ids=[])

,,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_name,app_version,,,,,
rag-documentation,base,0.333333,0.666667,0.5,3.362791,0.213734
RAG,simple,NaN,0.333333,NaN,0.609406,0.000000
My Retriever,base,NaN,0.166667,NaN,0.564138,0.000000
FOMC Retriever,base,NaN,0.000000,NaN,1.133732,0.000000


In [18]:
from trulens.dashboard import run_dashboard

run_dashboard(tru_session)

Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://10.0.1.18:56687 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [40]:
with tru_rag as recording:
    myrag.query(
        "How to make chocolate milk?"
    )
    myrag.query(
        "How to make pina colada"
    )
    myrag.query("How to make a macchiato?")

calling <function ragclass.query at 0x77c027741080> with (<__main__.ragclass object at 0x77c02d9374d0>, 'How to make chocolate milk?')
calling <function ragclass.retrieve at 0x77c027741d00> with (<__main__.ragclass object at 0x77c02d9374d0>,)
{'results': [{'chunk': 'milk, and melted butter in a microwave-safe mug. Sprinkle chocolate chips on top. Microwave for 1-2 minutes until cooked. Serve warm. Cuisine: General Diet Type: Vegetarian  \\nRecipe ID: 1539 Recipe Name: Caramelized Banana Ingredients: Banana, brown sugar, butter Quantities (for 1 person): Banana: 1 (sliced lengthwise), brown sugar: 1 tsp, butter: 1/2 tsp Cooking Time: 10 minutes Steps: Heat butter in a pan, sprinkle brown sugar, and place banana slices. Cook until caramelized. Serve warm. Cuisine: General Diet Type: Vegetarian  \\nRecipe ID: 1540 Recipe Name: Strawberry Cream Cup Ingredients: Fresh strawberries, whipped cream, sugar Quantities (for 1 person): Strawberries: 1/2 cup (sliced), whipped cream: 1/4 cup, sugar:

Error occurred while parsing response: Expecting value: line 1 column 2 (char 1)
Error occurred while parsing response: Expecting value: line 1 column 2 (char 1)
Error calling wrapped function generate_completion.
json.decoder.JSONDecodeError: Expecting value: line 1 column 2 (char 1)

Error calling wrapped function query.
json.decoder.JSONDecodeError: Expecting value: line 1 column 2 (char 1)



JSONDecodeError: Expecting value: line 1 column 2 (char 1)

---

# Implementing

In [48]:
from snowflake.cortex import complete

class RAG:

    @instrument
    def retrieve(self, query: str, category: str) -> list:
        """Search for similar chunks based on query and category."""
        if category == "ALL":
            response = svc.search(query, COLUMNS, limit=NUM_CHUNKS)
        else:
            filter_obj = {"@eq": {"category": category}}
            response = svc.search(query, COLUMNS, filter=filter_obj, limit=NUM_CHUNKS)
        
        json_data = json.loads(response.model_dump_json())
        relative_paths = set(item.get('relative_path', '') for item in json_data['results'])

        if response.results:
            retrieved_chunks = [curr["chunk"] for curr in response.results]

            return retrieved_chunks, relative_paths
        

    def create_prompt(query, category, prompt_context):
        """Create a prompt for the LLM with context from search results and chat history."""

        
        prompt = f"""
        I am Ali, a friendly and witty chef who specializes in {category} recipes! I love helping people cook and finding the perfect recipes from our collection.

        Conversation Flow:
        1. When suggesting recipes:
            - Prioritize recipes that makes use of all ingredients
            - First list all matching recipes as numbered options
            - Ask which recipe they'd like to know more about
        2. When user selects a recipe, provide full details in this format:
            Recipe Name:
            Quantities (for 1 person):
            Cooking Time:
            Steps:
            Cuisine:
            General Diet Type:

        <context>
        {prompt_context}
        </context>

        User Query: {query}
        Current Category: {category}

        Response (as Ali, friendly and category-aware):
        """

        return prompt
    
    @instrument
    def generate_completion(self, query: str, prompt: str, context_rag: list) -> str:

        return complete("mistral-large2", prompt)
    
    @instrument
    def query(self, query: str, category: str):
        context_rag, relative_paths = self.retrieve(query=query, category=category)
        prompt = self.create_prompt(category=category, prompt_context=context_rag)
        completion = self.generate_completion(
            query=query, prompt=prompt, context_rag=context_rag
        )

        return completion, relative_paths

decorating <function RAG.retrieve at 0x7e2ff136a160>
decorating <function RAG.generate_completion at 0x7e2ff1368ae0>
decorating <function RAG.query at 0x7e2ff1368680>
adding method <class '__main__.RAG'> retrieve __main__
adding method <class '__main__.RAG'> generate_completion __main__
adding method <class '__main__.RAG'> query __main__


In [49]:
myrag_imp = RAG()

In [50]:
out = myrag_imp.query(query="How to make pina colada", category="Juices")
out

(" Hello there! I'm Ali, your friendly and witty chef specializing in juice recipes. I'm here to help you find the perfect juice recipe from our collection. Let's get started!\n\nBased on the ingredients you have, here are some delicious juice recipes you might enjoy:\n\n1. **Pineapple Coconut Cooler**\n2. **Mixed Berry Sparkler**\n3. **Honeydew Mint Cooler**\n4. **Pineapple Mint Cooler**\n5. **Papaya Lime Twist**\n6. **Cantaloupe Honeydew Cooler**\n7. **Kiwi Strawberry Splash**\n8. **Tropical Splash**\n9. **Green Apple Spinach Detox**\n10. **Pear Ginger Twist**\n11. **Pomegranate Citrus**\n\nWhich recipe would you like to know more about?",
 {'juices.pdf'})

In [36]:
# from functools import partial
from trulens.providers.cortex.provider import Cortex
from trulens.core import Feedback
from trulens.core import Select
# import numpy as np

provider = Cortex(session, "mistral-large2")

# Define a groundedness feedback function
f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons, name="Groundedness"
    )
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
)
# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name="Answer Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)

# Context relevance between question and each context chunk.
f_context_relevance = (
    Feedback(
        provider.context_relevance_with_cot_reasons, name="Context Relevance"
    )
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)  # choose a different aggregation method if you wish
)

✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.app.retrieve.args.query .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.app.retrieve.args.query .
✅ In Context Relevance, input context will be set to __record__.app.retrieve.rets.collect() .


In [37]:
tru_rag = TruCustomApp(
    myrag_imp,
    app_name="rag-implement",
    app_version="base",
    feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance],
)

Function <function RAG.generate_completion at 0x7e2ff2450e00> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG object at 0x7e3009620fd0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG.query at 0x7e2ff2450a40> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG object at 0x7e3009620fd0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG.generate_completion at 0x7e300c2ce660> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG object at 0x7e3009620fd0> or provide a bound method for it as TruCustomApp constructor argument `methods_to_instrument`.
Function <function RAG.retrieve at 0x7e2ff1502ca0> was not found during instrumentation walk. Make sure it is accessible by traversing app <__main__.RAG object

instrumenting <class '__main__.RAG'> for base <class '__main__.RAG'>
	instrumenting retrieve
	instrumenting generate_completion
	instrumenting query
skipping base <class 'object'> because of class


In [44]:
with tru_rag as recording:
    m,n = myrag_imp.query(query="How to make a hazelnuts milk shake", category="Juices")

calling <function RAG.query at 0x7e2ff158d620> with (<__main__.RAG object at 0x7e3009620fd0>,)
calling <function RAG.retrieve at 0x7e2ff158e340> with (<__main__.RAG object at 0x7e3009620fd0>,)


calling <function RAG.generate_completion at 0x7e2ff158ed40> with (<__main__.RAG object at 0x7e3009620fd0>,)


Unsure what the main input string is for the call to query with args <BoundArguments (self=<__main__.RAG object at 0x7e3009620fd0>, query='How to make a hazelnuts milk shake', category='Juices')>.


/home/vscode/.local/lib/python3.11/site-packages/trulens/feedback/llm_provider.py:1521: UserWarning: Failed to process and remove trivial statements. Proceeding with all statements.
  warnings.warn(
/home/vscode/.local/lib/python3.11/site-packages/trulens/feedback/llm_provider.py:286: UserWarning: No supporting evidence provided. Returning score only.
  warnings.warn(


In [47]:
m

" Hello there! I'm Ali, your friendly and witty chef specializing in juice recipes. I'm here to help you find the perfect juice recipe from our collection. Let's get started!\n\nBased on the ingredients you have, here are some matching recipes:\n\n1. **Lime, Almond Milk**\n2. **Carrot Beetroot Detox**\n3. **Cantaloupe Mint Cooler**\n4. **Grapefruit Ginger Refresh**\n5. **Carrot Lemonade**\n6. **Pineapple Coconut Breeze**\n7. **Tropical Green Juice**\n8. **Strawberry Banana Bliss**\n9. **Coconut Lime Cooler**\n10. **Pear Pineapple Sparkle**\n\nWhich recipe would you like to know more about?"

In [39]:
tru_session.get_leaderboard()

,,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_name,app_version,,,,,
rag-documentation,base,0.333333,0.666667,0.500000,3.362791,0.213734
rag-implement,base,0.000000,0.166667,0.592262,6.016431,0.548322
RAG,simple,NaN,0.333333,NaN,0.609406,0.000000
My Retriever,base,NaN,0.166667,NaN,0.564138,0.000000
FOMC Retriever,base,NaN,0.000000,NaN,1.133732,0.000000


In [28]:
from trulens.dashboard import run_dashboard

run_dashboard(tru_session)

Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://10.0.0.70:60959 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [20]:
print(Select.RecordCalls.retrieve.args['query'])

__record__.app.retrieve.args.query


In [18]:
Select.RecordCalls.retrieve.args.

{'$defs': {'Step': {'description': 'A step in a selection path.',
   'properties': {},
   'title': 'Step',
   'type': 'object'}},
 'description': "Lenses into python objects.\n\nExample:\n    ```python\n    path = Lens().record[5]['somekey']\n\n    obj = ... # some object that contains a value at `obj.record[5]['somekey]`\n\n    value_at_path = path.get(obj) # that value\n\n    new_obj = path.set(obj, 42) # updates the value to be 42 instead\n    ```\n\n## `collect` and special attributes\n\nSome attributes hold special meaning for lenses. Attempting to access\nthem will produce a special lens instead of one that looks up that\nattribute.\n\nExample:\n    ```python\n    path = Lens().record[:]\n\n    obj = dict(record=[1, 2, 3])\n\n    value_at_path = path.get(obj) # generates 3 items: 1, 2, 3 (not a list)\n\n    path_collect = path.collect()\n\n    value_at_path = path_collect.get(obj) # generates a single item, [1, 2, 3] (a list)\n    ```",
 'properties': {'path': {'items': {'$ref': 

In [22]:
from trulens.core.schema import Record

tru_rag.

<bound method App.main_input of TruCustomApp(tru_class_info=trulens.apps.custom.TruCustomApp, app_id='app_hash_103966f93ac981b74dc5a1dab3ed3010', app_name='rag-implement', app_version='base', tags='-', metadata={}, feedback_definitions=['feedback_definition_hash_1d75df48597ad62a3cd354692947a811', 'feedback_definition_hash_8455e3af53c81938dc92a0a84fa93374', 'feedback_definition_hash_dc277695a26ae30d33570140c36db321'], feedback_mode=<FeedbackMode.WITH_APP_THREAD: 'with_app_thread'>, record_ingest_mode=<RecordIngestMode.IMMEDIATE: 'immediate'>, root_class=__main__.RAG, app=<__main__.RAG object at 0x7e300c2eca50>, initial_app_loader_dump=None, app_extra_json={}, feedbacks=[FeedbackDefinition(Groundedness,
	selectors={'source': Lens().__record__.app.retrieve.rets.collect(), 'statement': Lens().__record__.main_output},
	if_exists=None
), FeedbackDefinition(Answer Relevance,
	selectors={'prompt': Lens().__record__.app.retrieve.args.query, 'response': Lens().__record__.main_output},
	if_exists

In [23]:
context = Select.RecordCalls.retrieve.rets.rets[:]

In [27]:
context.model_computed_fields

{}